i have this aim from this analysis. first of all
What type of posts are engaging? - Engagement Analysis
What are the major topics for these posts? - Topic Analysis
Which topics are the most and least engaging?

to this i scrap different sources for burgerme for example the facebook and also the google forum.

*   Create an engagement scores by taking weighted average of likes and comments for the posts
*   Identify the topics for top and bottom quartile

In [ ]:
# load the data
import pandas as pd
Nf_1 = pd.read_csv("/content/facebook_Kiel.csv")
Nf_1.columns
Nf_1.drop(['link', 'share', 'time'], axis=1, inplace= True )

Nf_2 = pd.read_csv("/content/google (1).csv")
Nf_2.drop(['X8zlde', 'JKPXdf', 'JKPXdf href', 'KgoSse', 'KgoSse href', 'YLpWtd', 'e7TeLd href', 'rbm24b' ], axis=1, inplace= True )

#  attach two table together and fill the null in the comments column.
data=pd.concat([Nf_1, Nf_2], axis=0)
data['comments 1'].fillna('ist', inplace=True)
data['comments 2'].fillna('ist', inplace=True)
data['post description 1'].fillna('ist', inplace=True)
data['post description 2'].fillna('ist', inplace=True)
data['post description 3'].fillna('ist', inplace=True)
data['post description 4'].fillna('ist', inplace=True)
data['Vz1Vkc'].fillna('ist', inplace=True)
data['QRCoQb'].fillna('0', inplace=True)
data['like'].fillna('0', inplace=True)

Since we need english text for topic modeling,
i creat english comment columns:

In [ ]:
from googletrans import Translator

translator = Translator()
data['comments 1_en'] = data['comments 1'].apply(lambda x: translator.translate(x, src='de', dest='en').text if x is not None else "")
data['comments 2_en'] = data['comments 2'].apply(lambda x: translator.translate(x, src='de', dest='en').text if x is not None else "")
data['post description 1_en'] = data['post description 1'].apply(lambda x: translator.translate(x, src='de', dest='en').text if x is not None else "")
data['post description 2_en'] = data['post description 2'].apply(lambda x: translator.translate(x, src='de', dest='en').text if x is not None else "")
data['post description 3_en'] = data['post description 3'].apply(lambda x: translator.translate(x, src='de', dest='en').text if x is not None else "")
data.head()

**Doing preprocessing NLP methods:**


*   **Tokenization**
*   **Removing Stopwords and Special Characters (Using re)**
*   **Lemmatization**


For first analysis we use the german pre-trained model.

In [ ]:
!python -m spacy download de_core_news_sm

In [ ]:
# tokenize
import spacy
# load a pretrained german model for our anlaysis
nlp = spacy.load("de_core_news_sm")
# use our model for column comment 1
data['comments 1'] = data['comments 1'].apply(lambda x: nlp(x))
data['comments 2'] = data['comments 2'].apply(lambda x: nlp(x))
data['Vz1Vkc']= data['Vz1Vkc'].apply(lambda x: nlp(x))

#remove stopwords from comments
import nltk as n
from nltk.corpus import stopwords  # Import stopwords
n.download('stopwords')

# Modified to handle spaCy Doc objects
data['comments 1'] = data['comments 1'].apply(lambda comment: ' '.join([token.text for token in comment if token.text.lower() not in set(stopwords.words('german'))]))
data['comments 2'] = data['comments 2'].apply(lambda comment: ' '.join([token.text for token in comment if token.text.lower() not in set(stopwords.words('german'))]))
data['Vz1Vkc'] = data['Vz1Vkc'].apply(lambda comment: ' '.join([token.text for token in comment if token.text.lower() not in set(stopwords.words('german'))]))

#remove sth like ! from comments
import re
data['comments 1'] = data['comments 1'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
data['comments 2'] = data['comments 2'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
data['Vz1Vkc'] = data['Vz1Vkc'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

# Nf_1['comments 1'] now contains strings,
# we need to convert them back to spaCy Doc objects
data['comments 1'] = data['comments 1'].apply(lambda text: nlp(text))
data['comments 2'] = data['comments 2'].apply(lambda text: nlp(text))
data['Vz1Vkc'] = data['Vz1Vkc'].apply(lambda text: nlp(text))
# Now we can perform lemmatization
data['comments 1'] = data['comments 1'].apply(lambda doc: ' '.join([token.lemma_ for token in doc]))
data['comments 2'] = data['comments 2'].apply(lambda doc: ' '.join([token.lemma_ for token in doc]))
data['Vz1Vkc'] = data['Vz1Vkc'].apply(lambda doc: ' '.join([token.lemma_ for token in doc]))



# *Analysis*
## Basic Sentiment
you can use sentiment analysis to gauge the general positivity or negativity of each comment and combine that with engagement metrics like the number of likes to create a scoring system.

In [ ]:
import numpy as np
from textblob import TextBlob
# define a function for engagment comments text
def sentimentfunction(text):
    blob = TextBlob(text)
    #return 0, 1, -1 score
    return blob.sentiment.polarity

data['sentiment_score'] = data['comments 1'].apply(sentimentfunction)
data['sentiment_score 2'] = data['comments 2'].apply(sentimentfunction)
data['sentiment_score 3'] = data['Vz1Vkc'].apply(sentimentfunction)
data[ 'avarage_sentiment_score'] = data[['sentiment_score', 'sentiment_score 2', 'sentiment_score 3']].mean(axis=1)
data['like'] = pd.to_numeric(data['like'], errors='coerce')
data['like'] = data['like'].fillna(0)
data['sentiment_score'] = data['avarage_sentiment_score'].fillna(0)
data['engagment score']= data['like']*data['avarage_sentiment_score']

In [ ]:
sentiment_score = data[data['avarage_sentiment_score'] != 0]
sentiment_score["avarage_sentiment_score"]

The result will be sth like


```
avarage_sentiment_score
3	0.266667
35	0.111111
0	0.177778
1	0.048810
2	-0.083983
.
.
.
```



In [ ]:
engagment_score = data[data['engagment score'] != 0]
engagment_score["engagment score"]


The result will be sth like


```
engagment score
3	0.830000
7 0.112000
.
.
.
```
and by this we find Tone of voice is positive or negative.


# Advanced Analysis

In [ ]:
!python -m spacy download en_core_web_md

In [ ]:
# eleminate the stopwords # i use only post description 1 and 2 for topic clustring since i relize that 3 is meangless
# use our model for column comment 1
nlpen=spacy.load("en_core_web_md")
data['comments 1_en'] = data['comments 1_en'].apply(lambda x: nlpen(x))
data['comments 2_en'] = data['comments 2_en'].apply(lambda x: nlpen(x))
data['post description 1_en'] = data['post description 1_en'].apply(lambda x: nlpen(x))
data['post description 2_en'] = data['post description 2_en'].apply(lambda x: nlpen(x))
from nltk.corpus import stopwords  # Import stopwords
n.download('stopwords')

# Modified to handle spaCy Doc objects
data['comments 1_en'] = data['comments 1_en'].apply(lambda comment: ' '.join([token.text for token in comment if token.text.lower() not in set(stopwords.words('english'))]))
data['comments 2_en'] = data['comments 2_en'].apply(lambda comment: ' '.join([token.text for token in comment if token.text.lower() not in set(stopwords.words('english'))]))
data['post description 1_en'] = data['post description 1_en'].apply(lambda comment: ' '.join([token.text for token in comment if token.text.lower() not in set(stopwords.words('english'))]))
data['post description 2_en'] = data['post description 2_en'].apply(lambda comment: ' '.join([token.text for token in comment if token.text.lower() not in set(stopwords.words('english'))]))

#remove sth like ! from comments
import re
data['comments 1_en'] = data['comments 1_en'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
data['comments 2_en'] = data['comments 2_en'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
data['post description 1_en'] = data['post description 1_en'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
data['post description 2_en'] = data['post description 2_en'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

# lemmit.
data['comments 1_en'] = data['comments 1_en'].apply(lambda text: nlp(text))
data['comments 2_en'] = data['comments 2_en'].apply(lambda text: nlp(text))
data['post description 1_en'] = data['post description 1_en'].apply(lambda text: nlp(text))
data['post description 2_en'] = data['post description 2_en'].apply(lambda text: nlp(text))
# Now we can perform lemmatization
data['comments 1_en'] = data['comments 1_en'].apply(lambda doc: ' '.join([token.lemma_ for token in doc]))
data['comments 2_en'] = data['comments 2_en'].apply(lambda doc: ' '.join([token.lemma_ for token in doc]))
data['post description 1_en'] = data['post description 1_en'].apply(lambda doc: ' '.join([token.lemma_ for token in doc]))
data['post description 2_en'] = data['post description 2_en'].apply(lambda doc: ' '.join([token.lemma_ for token in doc]))



#retokenize it again to use embedding
data['comments 1_en'] = data['comments 1_en'].apply(lambda x: nlpen(x))
data['comments 2_en'] = data['comments 2_en'].apply(lambda x: nlpen(x))
data['post description 1_en'] = data['comments 2_en'].apply(lambda x: nlpen(x))
data['post description 2_en'] = data['comments 2_en'].apply(lambda x: nlpen(x))


Use **k-mean** method for topic modeling

In [ ]:
import spacy
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
nlpen=spacy.load("en_core_web_md")
#combine the text of comments
data['commenpost_combined'] = data['comments 1_en'].apply(lambda doc: doc.text) + ' ' + data['comments 2_en'].apply(lambda doc: doc.text)+ ' ' + data['post description 1_en'].apply(lambda doc: doc.text) + ' ' + data['post description 2_en'].apply(lambda doc: doc.text)
# do embedding
data['embeding_comment']= data['commenpost_combined'].apply(lambda x:nlpen(x).vector)
embedding_matrix=data['embeding_comment'].to_list()


In [ ]:
# define a range of number of clustering group a range between 1 to 10
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
# Initialize inertias as an empty list to store inertia values
inertias = []
K_v= range(1, 10)
for k in K_v:
  kmeans = KMeans(n_clusters=k, random_state=0).fit(embedding_matrix)
  inertias.append(kmeans.inertia_)

plt.plot(K_v, inertias, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.title('Elbow Method For Optimal K')
plt.show()

By our **Elbow diagram** i realize that from 3 or 4 cluster group we have no especial inertia decreasing and means that i choose the number of clustering as 4.

In [ ]:
#doing final k-means analysis
numcluster = 4
kmeans = KMeans(n_clusters=numcluster, random_state=0).fit(embedding_matrix)
data['cluster'] = kmeans.labels_

In [ ]:
# Reduce dimensionality for visualization
pca = PCA(n_components=2)
reduced_embeddings = pca.fit_transform(embedding_matrix)

plt.figure(figsize=(10, 7))
plt.scatter(reduced_embeddings[:, 0], reduced_embeddings[:, 1], c=data['cluster'], cmap='viridis')
plt.colorbar()
plt.title("Post Topics Clustering")
plt.xlabel("PCA Component 1")
plt.ylabel("PCA Component 2")
plt.show()

In [ ]:
data['cluster'].value_counts()

the result will be sth like this


```

cluster
0	n_1
1	n_2
2	n_3
3	n_4
```




In [ ]:
#form green here i mean purple group because i have correct a mistake and the color changed. but i have not change the code name.
green_label =0
green_post = data[data['cluster']== green_label]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import matplotlib.pyplot as plt
import seaborn as sns

### Verctorization

In [ ]:
vectorizer= CountVectorizer(stop_words = 'english', max_features=5000)
#Document-Term Matrix (DTM)
X= vectorizer.fit_transform(green_post['commenpost_combined'])

### Topic modeling
by different topic number

In [ ]:
n_topics = 3
lda = LatentDirichletAllocation(n_components=n_topics, random_state=0)
lda.fit(X)

In [ ]:
def display_topics(model, feature_name, no_top_words):
    topics = {}
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        topic_keywords = [feature_name[i] for i in topic.argsort()[:-no_top_words - 1:-1]] # Fixed indentation, and change feature_names to feature_name
        print(" ".join(topic_keywords))
        topics["Topic %d:" % (topic_idx)] = " ".join(topic_keywords)
    return topics

In [ ]:
# recive the names from vectorize
feature_names = vectorizer.get_feature_names_out()
topics = display_topics(lda, feature_names, 10)

the result will be sth like this


```
Topic 0:
unfortunately time come silke yes storeen self shame larissa hope
.
.
.
```



In [ ]:
n_topics = 5
lda = LatentDirichletAllocation(n_components=n_topics, random_state=0)
lda.fit(X)

feature_names = vectorizer.get_feature_names_out()
topics = display_topics(lda, feature_names, 10)

n_topics = 10
lda = LatentDirichletAllocation(n_components=n_topics, random_state=0)
lda.fit(X)

feature_names = vectorizer.get_feature_names_out()
topics = display_topics(lda, feature_names, 10)

top_comments = green_post.nlargest(10, 'engagment score')
print("Top 10 Comments by Engagement in purple Cluster:")
print(top_comments[['commenpost_combined', 'engagment score']])

from wordcloud import WordCloud

fig, axes = plt.subplots(1, n_topics, figsize=(15, 5))
for i, topic in enumerate(topics.keys()):
    wordcloud = WordCloud(width=800, height=400, background_color="white").generate(" ".join(topics[topic]))
    axes[i].imshow(wordcloud, interpolation='bilinear')
    axes[i].axis("off")
    axes[i].set_title(f"{topic}")
plt.show()

# **1. Topic Analysis and Interpretation**
by the relation of topic cluster kezwords and engagment score we can intrpret the social media.



*  **High Engagement Areas**
*  **Customer Sentiment**

